In [ ]:
# Setup: install Qiskit (runs automatically in Colab, no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc

# Erstellen eines Pass Managers für Dynamical Decoupling

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Paketversionen</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen die Verwendung dieser oder neuerer Versionen.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Diese Seite demonstriert, wie der [`PadDynamicalDecoupling`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.PadDynamicalDecoupling)-Pass verwendet wird, um eine Fehlerunterdrückungstechnik namens _Dynamical Decoupling_ zur Schaltung hinzuzufügen.

Dynamical Decoupling funktioniert, indem Pulssequenzen (bekannt als _Dynamical Decoupling-Sequenzen_) zu inaktiven Qubits hinzugefügt werden, um sie um die Bloch-Kugel zu drehen, was die Wirkung von Rauschkanälen aufhebt und dadurch Dekohärenz unterdrückt. Diese Pulssequenzen ähneln Refokussierungspulsen, die in der Kernspinresonanz verwendet werden. Eine vollständige Beschreibung finden Sie in [A Quantum Engineer's Guide to Superconducting Qubits](https://arxiv.org/abs/1904.06560).

Da der `PadDynamicalDecoupling`-Pass nur auf geplanten Schaltungen arbeitet und Gates umfasst, die nicht notwendigerweise Basis-Gates unseres Ziels sind, benötigen Sie auch die [`ALAPScheduleAnalysis`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.ALAPScheduleAnalysis)- und [`BasisTranslator`](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.BasisTranslator)-Passes.

Dieses Beispiel verwendet `ibm_fez`, das zuvor initialisiert wurde. Holen Sie die `target`-Informationen vom `backend` und speichern Sie die Operationsnamen als `basis_gates`, da das `target` modifiziert werden muss, um Timing-Informationen für die im Dynamical Decoupling verwendeten Gates hinzuzufügen.

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService()
backend = service.backend("ibm_fez")

target = backend.target
basis_gates = list(target.operation_names)

Erstellen Sie als Beispiel eine `efficient_su2`-Schaltung. Transpilieren Sie zunächst die Schaltung zum Backend, da Dynamical Decoupling-Pulse hinzugefügt werden müssen, nachdem die Schaltung transpiliert und geplant wurde. Dynamical Decoupling funktioniert oft am besten, wenn es viel Leerlaufzeit in den Quantenschaltungen gibt - das heißt, es gibt Qubits, die nicht verwendet werden, während andere aktiv sind. Dies ist bei dieser Schaltung der Fall, da die Zwei-Qubit-`ecr`-Gates in diesem Ansatz sequenziell angewendet werden.

In [2]:
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.circuit.library import efficient_su2

qc = efficient_su2(12, entanglement="circular", reps=1)
pm = generate_preset_pass_manager(1, target=target, seed_transpiler=12345)
qc_t = pm.run(qc)
qc_t.draw("mpl", fold=-1, idle_wires=False)

<Image src="../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/8228f889-806a-4873-b1da-27c9795d5f5c-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/8228f889-806a-4873-b1da-27c9795d5f5c-0.svg)

Eine Dynamical Decoupling-Sequenz ist eine Reihe von Gates, die zur Identität zusammengesetzt werden und in der Zeit regelmäßig verteilt sind. Beginnen Sie beispielsweise mit der Erstellung einer einfachen Sequenz namens XY4, die aus vier Gates besteht.

In [3]:
from qiskit.circuit.library import XGate, YGate

X = XGate()
Y = YGate()

dd_sequence = [X, Y, X, Y]

Aufgrund der regelmäßigen Zeitsteuerung von Dynamical Decoupling-Sequenzen müssen Informationen über das `YGate` zum `target` hinzugefügt werden, da es *kein* Basis-Gate ist, während das `XGate` eines ist. Wir wissen jedoch *a priori*, dass das `YGate` die gleiche Dauer und den gleichen Fehler wie das `XGate` hat, sodass wir diese Eigenschaften einfach vom `target` abrufen und für die `YGate`-Objekte zurückgeben können. Deshalb wurden die `basis_gates` separat gespeichert, da wir die `YGate`-Anweisung zum `target` hinzufügen, obwohl es kein tatsächliches Basis-Gate von `ibm_fez` ist.

In [4]:
from qiskit.transpiler import InstructionProperties

y_gate_properties = {}
for qubit in range(target.num_qubits):
    y_gate_properties.update(
        {
            (qubit,): InstructionProperties(
                duration=target["x"][(qubit,)].duration,
                error=target["x"][(qubit,)].error,
            )
        }
    )

target.add_instruction(YGate(), y_gate_properties)

Ansatz-Schaltungen wie `efficient_su2` sind parametrisiert, sodass ihnen Werte zugewiesen werden müssen, bevor sie an das Backend gesendet werden. Hier werden zufällige Parameter zugewiesen.

In [5]:
import numpy as np

rng = np.random.default_rng(1234)
qc_t.assign_parameters(
    rng.uniform(-np.pi, np.pi, qc_t.num_parameters), inplace=True
)

Als Nächstes führen Sie die benutzerdefinierten Passes aus. Instanziieren Sie den `PassManager` mit `ALAPScheduleAnalysis` und `PadDynamicalDecoupling`. Führen Sie zuerst `ALAPScheduleAnalysis` aus, um Timing-Informationen über die Quantenschaltung hinzuzufügen, bevor die regelmäßig verteilten Dynamical Decoupling-Sequenzen hinzugefügt werden können. Diese Passes werden mit `.run()` auf der Schaltung ausgeführt.

In [6]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes.scheduling import (
    ALAPScheduleAnalysis,
    PadDynamicalDecoupling,
)

dd_pm = PassManager(
    [
        ALAPScheduleAnalysis(target=target),
        PadDynamicalDecoupling(target=target, dd_sequence=dd_sequence),
    ]
)
qc_dd = dd_pm.run(qc_t)

Verwenden Sie das Visualisierungstool [`timeline_drawer`](https://docs.quantum.ibm.com/api/qiskit/qiskit.visualization.timeline_drawer), um die Zeitsteuerung der Schaltung zu sehen und zu bestätigen, dass eine regelmäßig verteilte Sequenz von `XGate`-Objekten und `YGate`-Objekten in der Schaltung erscheint.

In [7]:
from qiskit.visualization import timeline_drawer

timeline_drawer(qc_dd, idle_wires=False, target=target)

<Image src="../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/cb73e2c4-ab05-4f15-91ae-2fab64028d6e-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/cb73e2c4-ab05-4f15-91ae-2fab64028d6e-0.svg)

Wenden Sie zuletzt manuell den `BasisTranslator`-Pass an, da das `YGate` kein tatsächliches Basis-Gate unseres Backends ist (dies ist ein Standard-Pass, wird aber vor der Planung ausgeführt, sodass er erneut angewendet werden muss). Die Session Equivalence Library ist eine Bibliothek von Schaltungsäquivalenzen, die es dem Transpiler ermöglicht, Schaltungen in Basis-Gates zu zerlegen, wie ebenfalls als Argument angegeben.

In [8]:
from qiskit.circuit.equivalence_library import (
    SessionEquivalenceLibrary as sel,
)
from qiskit.transpiler.passes import BasisTranslator

qc_dd = BasisTranslator(sel, basis_gates)(qc_dd)
qc_dd.draw("mpl", fold=-1, idle_wires=False)

<Image src="../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/aaa27ee4-1965-41bf-abd2-1d9176af6dc4-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/dynamical-decoupling-pass-manager/extracted-outputs/aaa27ee4-1965-41bf-abd2-1d9176af6dc4-0.svg)

Jetzt fehlen `YGate`-Objekte in unserer Schaltung, und es gibt explizite Timing-Informationen in Form von `Delay`-Gates. Diese transpilierte Schaltung mit Dynamical Decoupling ist nun bereit, an das Backend gesendet zu werden.

## Nächste Schritte

> **Tip:** - Um zu erfahren, wie Sie die Funktion `generate_preset_passmanager` anstelle des Schreibens eigener Passes verwenden, beginnen Sie mit dem Thema [Transpilation default settings and configuration options](defaults-and-configuration-options).
>     - Probieren Sie den Guide [Compare transpiler settings](/guides/circuit-transpilation-settings#compare-transpiler-settings) aus.
>     - Siehe die [Transpile-API-Dokumentation.](https://docs.quantum-computing.ibm.com/api/qiskit/transpiler)